In [ ]:
import sagemaker
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter
from sagemaker.xgboost.estimator import XGBoost

session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = session.default_bucket()

s3_input_path = "s3://ai-trading-copilot-curated/features/fe-yearly-run-2025-10-18-00-39-10/"

s3_output_path = f"s3://{default_bucket}/stock-model-tuning/"

estimator = XGBoost(
    entry_point="final_model.py",
    source_dir="model", 
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='1.7-1',
    output_path=s3_output_path,
    py_version='py3'
)

hyperparameter_ranges = {
    'max_depth': IntegerParameter(3, 8),
    'eta': ContinuousParameter(0.01, 0.2), 
    'min_child_weight': IntegerParameter(1, 10),
    'subsample': ContinuousParameter(0.5, 0.9),
    'colsample_bytree': ContinuousParameter(0.5, 0.9)
}

# The tuner will find the job that maximized the validation:auc metric
tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name='validation:auc',
    hyperparameter_ranges=hyperparameter_ranges,
    objective_type='Maximize',
    max_jobs=20,
    max_parallel_jobs=4,
    base_tuning_job_name='stock-xgb-tuning'
)

tuner.fit({'train': s3_input_path})

print("Tuning job launched! Monitor its progress in the SageMaker console.")

In [ ]:
import boto3


s3_uri = "s3://sagemaker-us-east-1-[ACCOUNT]/[TRAINED-MODEL]/output/model.tar.gz"

from urllib.parse import urlparse
parsed_uri = urlparse(s3_uri)
bucket_name = parsed_uri.netloc
s3_key = parsed_uri.path.lstrip('/')
local_filename = "best_model.tar.gz"

s3_client = boto3.client('s3')
print(f"Downloading {s3_uri} to {local_filename}...")
s3_client.download_file(bucket_name, s3_key, local_filename)
print(" Download complete.")

In [ ]:
!tar -xvzf best_model.tar.gz model.pkl